In [15]:
import os
import math
from typing import List

import matplotlib.pyplot as plt
import numpy as np

import copy


class Point():
    """Data structure of a point."""

    def __init__(self, x, y, z=0) -> None:
        """Point object
        Args:
            x (float): x coordinate of point
            y (float): y coordinate of point
        """
        self.x = x
        self.y = y
        self.z = z

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.z

    def __len__(self):
        return 3

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def to_numpy(self):
        return np.array([self.x, self.y])


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __eq__(self, other):
        return self.__dict__ == other.__dict__
        #if self.p1 == other.p1 or self.p1 == other.p2:
        #    if self.p2 == other.p1 or self.p2 == other.p2:
        #        return True
        #return False

    def __str__(self):
        return "[" + str(self.p1) + " -> " + str(self.p2) + "]"

    def __call__(self):
        # return "[" + str(self.p1) + " -> " + str(self.p2) + "]"
        return (self.p1(), self.p2())


class Face:
    def __init__(self, e1, e2, e3):
        self.e1 = e1
        self.e2 = e2
        self.e3 = e3

    def __str__(self):
        return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"

    def __call__(self):
        # return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"
        return self.e1.p1(), self.e1.p2(), self.e3.p1()

    def __eq__(self, other):
        p1, p2, p3 = self()
        p1_, p2_, p3_ = other()

        if p1 == p1_ or p1 == p2_ or p1 == p3_:
            if p2 == p1_ or p2 == p2_ or p2 == p3_:
                if p3 == p1_ or p3 == p2_ or p3 == p3_:
                    return True
        return False
        # if self.e1 == other.e1 or self.e1 == other.e2 or self.e2 == other.e3:
        #     if self.e2 == other.e1 or self.e2 == other.e2 or self.e2 == other.e3:
        #         if self.e3 == other.e1 or self.e3 == other.e2 or self.e3 == other.e3:
        #             return True
        # return False


class Gcrf():

    def gen_poly(self, n):
        xn = np.random.random(n)
        yn = np.random.random(n)
        z = zip(xn, yn)
        z = [Point(x, y) for x, y in z]
        z = np.array(z)
        angles = [self.pseudo_angulo_orientado(i) for i in z]
        indexes = np.argsort(angles)
        z = z[indexes]
        z = z.tolist()

        return z

    def soma_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x+b.x, a.y+b.y, a.z+b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]+b[i])
        return z

    def subtr_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x-b.x, a.y-b.y, a.z-b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]-b[i])
        return z

    def multi_escalar(self, lamb: int, x: list) -> list:
        # if isinstance(a, Point):
        #     return Point(a.x*lamb, a.y*lamb, a.z*lamb)
        z = []
        for i in range(len(x)):
            z.append(lamb*x[i])
        return z

    def prod_escalar(self, a, b):
        aIsPoint = isinstance(a, Point)
        bIsPoint = isinstance(b, Point)
        if aIsPoint and bIsPoint:
            return sum([a.x*b.x, a.y*b.y, a.z*b.z])
        if aIsPoint:
            a = a()
        if bIsPoint:
            b = b()
        res = []
        for i in range(len(a)):
            res.append(a[i]*b[i])
        return sum(res)

    def norma(self, x):
        z = 0
        for i in range(len(x)):
            z += x[i]**2
        return math.sqrt(z)

    def distancia(self, x, y):
        """Returns the distance between vectors x and y.
        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.
        Returns:
            [float]: [the distance between vectors x and y]
        """
        z = []
        for i in range(len(x)):
            z.append(x[i]-y[i])
        return self.norma(z)

    def angulo(self, x, y, degrees=False):
        """Returns the angle between vectors x and y.
        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.
            degrees (bool): if the result should be in degrees or radians, defaults to False.
        Returns:
            float: the angle between the two vectors
        """

        if not degrees:
            return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))*(180/math.pi)

    def angulo_orientado(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*math.acos(x[0]/self.norma(x))
        return mult*math.acos(x[0]/self.norma(x))*(180/math.pi)

    def pseudo_angulo(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*(1-(x[0]/self.norma(x)))
        return mult*(1-(x[0]/self.norma(x)))*(180/math.pi)

    def pseudo_angulo_cosseno(self, x, y=[0, 0], degrees=False):
        if isinstance(x, Point):
            x = x()
        if not degrees:
            return 1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        return (1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))))*(180/math.pi)

    def pseudo_angulo_orientado(self, x):
        if isinstance(x, Point):
            x = x()
        if x[1] >= 0:
            if x[0] >= 0:
                if x[0] >= x[1]:
                    return x[1]/x[0]
                return 2-(x[0]/x[1])
            if -x[0] <= x[1]:
                return 2+(-x[0]/x[1])
            return 4-(x[1]/-x[0])
        if x[0] < 0:
            if -x[0] >= -x[1]:
                return 4+(-x[1]/-x[0])
            return 6-(-x[0]/-x[1])
        if x[0] <= -x[1]:
            return 6+(x[0]/-x[1])
        return 8-(-x[1]/x[0])

    def pseudo_angulo_2(self, x, y):
        return self.pseudo_angulo_orientado(y)-self.pseudo_angulo_orientado(x)

    def prod_vetorial(self, a, b):
        if len(a) == 2:
            return a.x*b.y-a.y*b.x
        return [a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x]

    def intersect(self, a, b, c, d):
        ab = self.subtr_vetorial(b, a)
        ac = self.subtr_vetorial(c, a)
        ad = self.subtr_vetorial(d, a)
        ca = self.subtr_vetorial(a, c)
        cb = self.subtr_vetorial(b, c)
        cd = self.subtr_vetorial(d, c)

        # r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad)
        r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad) < 0
        # r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb)
        r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb) < 0
        # return (r1, r2), r1 < 0 and r2 < 0
        return r1 and r2

    def square_area(self, p1: Point, p2: Point, p3: Point):
        res = self.prod_vetorial(self.subtr_vetorial(
            p2, p1), self.subtr_vetorial(p3, p1))
        res = sum([i**2 for i in res])
        return 0.5*res

    def area(self, x, y):
        if len(x) == 2:
            return abs(self.prod_vetorial(x, y))
        return self.norma(self.prod_vetorial(x, y))

    ah def signed_volume(self, p1: Point, p2: Point, p3: Point, p4: Point):
        v1 = self.subtr_vetorial(p2, p1)
        # print(v1)
        v2 = self.subtr_vetorial(p3, p1)
        # print(v2)
        v3 = self.subtr_vetorial(p4, p1)
        # print(v3)
        res = self.prod_vetorial(v1, v2)
        # print(res)
        return self.prod_escalar(v3, res)/6

    def antiHorario(self, listaDePontos):
        res = 0
        for i in range(len(listaDePontos)-1):
            res += self.prod_vetorial(listaDePontos[i], listaDePontos[i+1])
        res += self.prod_vetorial(listaDePontos[0], listaDePontos[-1])
        return 0.5*res > 0

    def slope(self, p1: Point, p2: Point):
        return (p2.y - p1.y) / (p2.x - p1.x)

    def pip(self, p: Point, poly) -> bool:
        """ Determine if the point is in the polygon.
        Args:
            p -- a object of type Point(x, y)
            poly -- a list of tuples [(x, y), (x, y), ...]
        Returns:
            True if the point is in the path or is a corner or on the boundary"""

        num = len(poly)
        j = num - 1
        result = False
        for i in range(len(poly)):
            if (p.x == poly[i].x) and (p.y == poly[i].y):
                return True
            if ((poly[i].y > p.y) != (poly[j].y > p.y)):
                slope = self.slope(p, poly[i])
                if slope == 0:
                    return True
                if (slope < 0) != (poly[j].y < poly[i].y):
                    result = not result
            j = i
        return result

    def starPolygon(self, points, jumps):
        i = 0
        starredPolygon = []
        while i < jumps:
            for j in range(i, len(points), jumps):
                # print("Ponto adicionado j:",points[j]())
                starredPolygon.append(points[j])
            # print("Ponto adicionado i:",points[i]())
            starredPolygon.append(points[i])
            i += 1
        return starredPolygon

    def centeroidnp(self, points):
        length = len(points)
        tipoPoints = type(points[0])

        if tipoPoints == tuple:
            arr = np.array(points)
        elif tipoPoints == Point:
            arr = np.array([p.to_numpy() for p in points])

        sum_x = np.sum(arr[:, 0])
        sum_y = np.sum(arr[:, 1])
        return sum_x/length, sum_y/length

    def plotPoint(self, p: Point):
        plt.scatter(p.x, p.y)
        plt.show()

    def plotPoints(self, p: List[Point]):
        for point in p:
            self.plotPoint(point)
        plt.show()

    def plotPolygon(self, points, annotate=False):
        """Plot polygon from list of points
        Args:
            points (Point): list of Point objects indicating the polygon vertices
        """

        points.append(points[0])
        if type(points[0]) == Point:
            points = [p() for p in points]
        x, y = zip(*points)
        plt.grid(0.5)
        plt.plot(x, y, '-o')
        if annotate:
            plt.text(x[0], y[0], "p1")
            plt.text(x[1], y[1], "p2")
            plt.text(x[2], y[2], "p3")
        plt.show()

    def plotSegment(self, segments: List[Point], threeD=False) -> None:
        if threeD:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

        for segment in segments:
            segx = [p.x for p in segment]
            segy = [p.y for p in segment]
            if threeD:
                segz = [p.z for p in segment]
                ax.plot(segx, segy, segz)
            else:
                plt.plot(segx, segy)
        plt.show()

    def plotSquare(self):
        x = [1, 1, -1, -1, 1]
        y = [1, -1, -1, 1, 1]
        plt.plot(x, y)

    def plotAxis(self):
        plt.grid()
        plt.axhline(y=0, color='k')
        plt.axvline(x=0, color='k')


    # def plot_faces(self, F):


    def rotationIndex(self, p: Point, poly: list):
        # NOT WORKING
        j = len(poly)-1
        rotIndex = 0
        for i in range(len(poly)):
            ppi = self.subtr_vetorial(p, poly[j])
            ppi1 = self.subtr_vetorial(p, poly[i])
            rotIndex += self.pseudo_angulo_2(ppi, ppi1)
            j = i
        rotIndex *= 1/(2*math.pi)
        if rotIndex == 0:
            return False
        if abs(rotIndex) == 1:
            return True
        return rotIndex

In [33]:
points = []
for i in range(1000):
    a = np.random.random(1)
    b = np.random.random(1)
    c = np.random.random(1)
    
    points.append(Point(a[0],b[0],c[0]))

print(len(points))
H = GiftWrap(points)
print(len(points))
write_faces(H, points)


1000
1000
[<__main__.Face object at 0x000002060FB4C550>, <__main__.Face object at 0x000002060FB4CA60>, <__main__.Face object at 0x000002060FB4C700>, <__main__.Face object at 0x000002060FB4C880>, <__main__.Face object at 0x000002060FB4C790>, <__main__.Face object at 0x000002060FB4CEE0>, <__main__.Face object at 0x000002060F8AA280>, <__main__.Face object at 0x000002060F8AA3A0>, <__main__.Face object at 0x000002060F8AA460>, <__main__.Face object at 0x000002060F8AA6A0>, <__main__.Face object at 0x000002060F8AA7C0>, <__main__.Face object at 0x000002060F8AABE0>, <__main__.Face object at 0x000002060F8AA0D0>, <__main__.Face object at 0x000002060F8AD040>, <__main__.Face object at 0x000002060F8AD280>, <__main__.Face object at 0x000002060F8AD3A0>, <__main__.Face object at 0x000002060F8AD520>, <__main__.Face object at 0x000002060F8ADA60>, <__main__.Face object at 0x000002060F8ADB80>, <__main__.Face object at 0x000002060F8AAAC0>, <__main__.Face object at 0x000002060F8CF040>, <__main__.Face object a

In [42]:
for p in H:
    print("v {} {} {}".format(p.e1.p1.x,p.e1.p1.y,p.e1.p1.z))
    print("v {} {} {}".format(p.e1.p2.x,p.e1.p2.y,p.e1.p2.z))
    print("v {} {} {}".format(p.e2.p2.x,p.e2.p2.y,p.e2.p2.z))

v 0.5778345001967247 0.0004186791788486355 0.02579088819137665
v 0.985307832933602 0.0007893989019558401 0.15038319514733256
v 0.47797285727441297 0.003419524758466852 0.9800289305707254
v 0.985307832933602 0.0007893989019558401 0.15038319514733256
v 0.5778345001967247 0.0004186791788486355 0.02579088819137665
v 0.8185541927792049 0.057386782512130585 0.03748073768103655
v 0.47797285727441297 0.003419524758466852 0.9800289305707254
v 0.985307832933602 0.0007893989019558401 0.15038319514733256
v 0.9410829358328165 0.1558649802806723 0.9858553210458876
v 0.5778345001967247 0.0004186791788486355 0.02579088819137665
v 0.47797285727441297 0.003419524758466852 0.9800289305707254
v 0.053633303591189674 0.0024812100819394667 0.1671663112951577
v 0.985307832933602 0.0007893989019558401 0.15038319514733256
v 0.9410829358328165 0.1558649802806723 0.9858553210458876
v 0.9410809528936194 0.011674111835354606 0.5190902911466146
v 0.9410829358328165 0.1558649802806723 0.9858553210458876
v 0.477972857

v 0.8655460823017911 0.994064064094228 0.29087366053303276
v 0.11759736641614604 0.9930315410142784 0.06028226857864272
v 0.9318877818646922 0.8716042729610696 0.7340263768513743
v 0.6598680872031303 0.998640230442947 0.9236669754371319
v 0.9410829358328165 0.1558649802806723 0.9858553210458876
v 0.9401178148778153 0.9079169364948252 0.1961537018751115
v 0.929247187732329 0.9459685444104173 0.9629691209044393
v 0.9646800475015022 0.21378798829910872 0.3188089339114474
v 0.315330265348588 0.7807786156534835 0.9995183368074089
v 0.41146045178356494 0.9893371311076118 0.9693755868184062
v 0.050448920240629636 0.8335384290167233 0.9671600864378607
v 0.9352055059326831 0.917322800413669 0.18450750601310895
v 0.4783779652105329 0.960935444317714 0.11103552297617636
v 0.8934187306144226 0.9861261105245095 0.24594084693190832
v 0.9352055059326831 0.917322800413669 0.18450750601310895
v 0.646429327397384 0.8144118122944091 0.012411645545374661
v 0.3934481575646255 0.9449786607100557 0.041452904

In [5]:
def normal(p1, p2, p3):
    v1 = gc.subtr_vetorial(p1,p2) 
#     print("v1",v1)
    v2 = gc.subtr_vetorial(p1,p3)
#     print("v2",v2)
    return gc.prod_vetorial(v1,v2)

In [23]:
gc = Gcrf()

def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

def PivotAroundEdge(e, P):
    #print(e)
    p = P[-1]
    # print(p)
    area2 = gc.square_area(e.p1, e.p2, p)
    for p_ in range(len(P)-1):
        volume = gc.signed_volume(e.p1, e.p2, p, P[p_])
        #print("P",p)
        #print("P_", P[p_])
        #print("VOLUME", volume)
        #face_no_hull = checa_face(H, Face(e, Edge(e.p1, P[p_]), Edge(P[p_], e.p2)))
        #print("FACE NO HULL", face_no_hull)
        if volume > 0:# and not face_no_hull:
            p = P[p_]
        elif volume == 0:
            area2_ = gc.square_area(e.p1, e.p2, P[p_])
            #print(area2)
            #print(area2_)
            if area2_ > area2:# and not face_no_hull:
                p = P[p_]
                area2 = area2_
    #print("PRÓXIMO PONTO", p)
    return p

def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = p

    for r in P:
        if (q.z == r.z) and (q.y == r.y) and (q.x < r.x):
            q = r
    
    if (q == p):
        q = Point(p.x + 1, p.y, p.z)
        #print("Chamando PIVOT no Find Edge on Hull")
        q = PivotAroundEdge(Edge(p, q), P)

    return Edge(p, q)

def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

def NotProcessed(E, e):
    #print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

def GiftWrap(P):
    t = FindTriangleOnHull(P)
    #print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []
    v = 2
    while not len(Q) == 0:
        e = Q.pop(0)
        #print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            #print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P)
            #print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            #print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
#                 print("Aresta que gerou a face {}:".format(v), e)
                H.append(t)
                v+=1
                #print("{} entrou no Hull".format(t))
            e_ = Edge(e.p2, e.p1)
            Q.extend([e_, t.e2, t.e3])
            # Q.extend([e, t.e2, t.e3])
            #print("Q agora possui as arestas:")
            #for q in Q: print(q)
            Processed.append(e)
            #print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
            #for aresta in Processed: print(aresta)
        #else: print("Aresta JÁ PROCESSADA")
        
    return H

In [8]:
# p1 = Point(0.0, -1.0, -1.0)
# p2 = Point(0.0, 1.0, -1.0)
# p3 = Point(1, -1.0, -0.5)
# p4 = Point(1, 1.0, -0.5)
# p5 = Point(1, -1.0, 0.5)
# p6 = Point(1, 1.0, 0.5)
# p7 = Point(-0.0, -1.0, 1.0)
# p8 = Point(-0.0, 1.0, 1.0)
# p9 = Point(-1, -1.0, 0.5)
# p10 = Point(-1, 1.0, 0.5)
# p11 = Point(-1, -1.0, -0.5)
# p12 = Point(-1, 1.0, -0.5)
p1 = Point(0.0, -1.0, -1.0)
p2 = Point(0.0, 1.0, -1.0)
p3 = Point(0.866025, -1.0, -0.5)
p4 = Point(0.866025, 1.0, -0.5)
p5 = Point(0.866025, -1.0, 0.5)
p6 = Point(0.866025, 1.0, 0.5)
p7 = Point(-0.0, -1.0, 1.0)
p8 = Point(-0.0, 1.0, 1.0)
p9 = Point(-0.866025, -1.0, 0.5)
p10 = Point(-0.866025, 1.0, 0.5)
p11 = Point(-0.866025, -1.0, -0.5)
p12 = Point(-0.866025, 1.0, -0.5)
P = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12]
resultado = GiftWrap(P)
print(len(resultado))
print_faces(resultado, P)

Aresta que gerou a face 2: [(0.866025, -1.0, -0.5) -> (0.0, -1.0, -1.0)]
Aresta que gerou a face 3: [(0.866025, -1.0, 0.5) -> (0.866025, -1.0, -0.5)]
Aresta que gerou a face 4: [(0.0, -1.0, -1.0) -> (0.866025, -1.0, 0.5)]
Aresta que gerou a face 5: [(-0.0, -1.0, 1.0) -> (0.0, -1.0, -1.0)]
Aresta que gerou a face 6: [(0.0, 1.0, -1.0) -> (0.0, -1.0, -1.0)]
Aresta que gerou a face 7: [(0.866025, -1.0, -0.5) -> (0.0, 1.0, -1.0)]
Aresta que gerou a face 8: [(0.866025, 1.0, -0.5) -> (0.866025, -1.0, -0.5)]
Aresta que gerou a face 9: [(0.866025, -1.0, 0.5) -> (0.866025, 1.0, -0.5)]
Aresta que gerou a face 10: [(-0.0, -1.0, 1.0) -> (0.866025, -1.0, 0.5)]
Aresta que gerou a face 11: [(0.0, -1.0, -1.0) -> (-0.0, -1.0, 1.0)]
Aresta que gerou a face 12: [(0.866025, 1.0, 0.5) -> (0.866025, -1.0, 0.5)]
Aresta que gerou a face 13: [(-0.866025, -1.0, 0.5) -> (0.0, -1.0, -1.0)]
Aresta que gerou a face 14: [(-0.0, -1.0, 1.0) -> (0.866025, -1.0, -0.5)]
Aresta que gerou a face 15: [(-0.866025, 1.0, -0.5) 

In [11]:
p1 = Point(0, 2, 0)
p2 = Point(0, 0, 5)
p3 = Point(0, 0, -10)
p4 = Point(0, 0, 0)
p5 = Point(0.1, 0.1, 0.1)
p6 = Point(-3, 0, 0)
p7 = Point(0, -3, 0)
p8 = Point(0, 0, 1)
p9 = Point(0, 0, -4)
p10 = Point(4, 0, 0)

P = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))
print_faces(H, P)
write_faces(H, P, "obj/pontosPB.obj")

Aresta que gerou a face 2: [(0, 0, 5) -> (0, -3, 0)]
Aresta que gerou a face 3: [(-3, 0, 0) -> (0, 0, 5)]
Aresta que gerou a face 4: [(0, -3, 0) -> (-3, 0, 0)]
Aresta que gerou a face 5: [(4, 0, 0) -> (0, -3, 0)]
Aresta que gerou a face 6: [(0, 0, 5) -> (4, 0, 0)]
Aresta que gerou a face 7: [(-3, 0, 0) -> (0, 2, 0)]
Aresta que gerou a face 8: [(4, 0, 0) -> (0, 0, -10)]
{[(0, -3, 0) -> (0, 0, 5)], [(0, 0, 5) -> (-3, 0, 0)], [(-3, 0, 0) -> (0, -3, 0)]}
{[(0, 0, 5) -> (0, -3, 0)], [(0, -3, 0) -> (4, 0, 0)], [(4, 0, 0) -> (0, 0, 5)]}
{[(-3, 0, 0) -> (0, 0, 5)], [(0, 0, 5) -> (0, 2, 0)], [(0, 2, 0) -> (-3, 0, 0)]}
{[(0, -3, 0) -> (-3, 0, 0)], [(-3, 0, 0) -> (0, 0, -10)], [(0, 0, -10) -> (0, -3, 0)]}
{[(4, 0, 0) -> (0, -3, 0)], [(0, -3, 0) -> (0, 0, -10)], [(0, 0, -10) -> (4, 0, 0)]}
{[(0, 0, 5) -> (4, 0, 0)], [(4, 0, 0) -> (0, 2, 0)], [(0, 2, 0) -> (0, 0, 5)]}
{[(-3, 0, 0) -> (0, 2, 0)], [(0, 2, 0) -> (0, 0, -10)], [(0, 0, -10) -> (-3, 0, 0)]}
{[(4, 0, 0) -> (0, 0, -10)], [(0, 0, -10) -> (0

In [ ]:
P = [Point(-0.336541, 0.160475, 0.104546),Point(-0.339937, 0.159270, 0.100140),Point(-0.333872, 0.160992, 0.108320),Point(-0.342575, 0.158237, 0.096786),Point(-0.345751, 0.156172, 0.093338),Point(-0.348727, 0.154105, 0.090201),Point(-0.352129, 0.151352, 0.086899),Point(-0.354974, 0.148769, 0.084337),Point(-0.357988, 0.145671, 0.081884),Point(-0.359439, 0.144293, 0.080621),Point(-0.361952, 0.141194, 0.078944),Point(-0.363842, 0.138784, 0.077743),Point(-0.365530, 0.136374, 0.076853),Point(-0.367688, 0.133448, 0.075606),Point(-0.370184, 0.129488, 0.074576),Point(-0.372210, 0.126045, 0.073904),Point(-0.375100, 0.120880, 0.073129),Point(-0.377590, 0.115715, 0.072974),Point(-0.379541, 0.111584, 0.072913),Point(-0.381406, 0.107108, 0.073235),Point(-0.383339, 0.102115, 0.073821),Point(-0.384817, 0.097295, 0.074991),Point(-0.386293, 0.092475, 0.076162),Point(-0.387471, 0.087655, 0.077799),Point(-0.388096, 0.084211, 0.079301),Point(-0.388623, 0.080768, 0.080958),Point(-0.389100, 0.075948, 0.083682),Point(-0.389095, 0.071988, 0.086536),Point(-0.388908, 0.068890, 0.089052),Point(-0.388535, 0.065446, 0.092107),Point(-0.388048, 0.062348, 0.095090),Point(-0.387305, 0.059422, 0.098347),Point(-0.386249, 0.056839, 0.101842),Point(-0.384907, 0.053912, 0.106030),Point(-0.383164, 0.050986, 0.110840),Point(-0.381740, 0.048921, 0.114537),Point(-0.380146, 0.047370, 0.118125),Point(-0.378365, 0.045477, 0.122251),Point(-0.376359, 0.044272, 0.126233),Point(-0.374296, 0.043240, 0.130178),Point(-0.372166, 0.042722, 0.133859),Point(-0.369577, 0.042379, 0.138125),Point(-0.366890, 0.042034, 0.142546),Point(-0.364477, 0.042379, 0.146045),Point(-0.361751, 0.043067, 0.149783),Point(-0.359113, 0.044099, 0.153137),Point(-0.356218, 0.045305, 0.156766),Point(-0.353311, 0.046854, 0.160167),Point(-0.350191, 0.048748, 0.163651),Point(-0.347783, 0.050297, 0.166276),Point(-0.346144, 0.051592, 0.167890),Point(-0.294198, 0.051592, 0.248553),Point(-0.293408, 0.050297, 0.250714),Point(-0.292014, 0.048748, 0.253991),Point(-0.290133, 0.046854, 0.258273),Point(-0.288239, 0.045305, 0.262326),Point(-0.286133, 0.044099, 0.266463),Point(-0.284170, 0.043067, 0.270253),Point(-0.281896, 0.042379, 0.274281),Point(-0.279708, 0.042034, 0.277925),Point(-0.276795, 0.042379, 0.282202),Point(-0.273982, 0.042722, 0.286322),Point(-0.271512, 0.043240, 0.289786),Point(-0.268774, 0.044272, 0.293295),Point(-0.265980, 0.045477, 0.296769),Point(-0.262960, 0.047370, 0.300098),Point(-0.260352, 0.048921, 0.303033),Point(-0.257576, 0.050986, 0.305859),Point(-0.253919, 0.053912, 0.309436),Point(-0.250661, 0.056839, 0.312391),Point(-0.247915, 0.059422, 0.314798),Point(-0.245257, 0.062348, 0.316822),Point(-0.242743, 0.065446, 0.318498),Point(-0.240117, 0.068890, 0.320102),Point(-0.237903, 0.071988, 0.321314),Point(-0.235307, 0.075948, 0.322498),Point(-0.232629, 0.080768, 0.323191),Point(-0.230903, 0.084211, 0.323398),Point(-0.229277, 0.087655, 0.323449),Point(-0.227299, 0.092475, 0.323054),Point(-0.225623, 0.097295, 0.322194),Point(-0.223945, 0.102115, 0.321334),Point(-0.222612, 0.107108, 0.319816),Point(-0.221548, 0.111584, 0.318252),Point(-0.220797, 0.115715, 0.316450),Point(-0.219907, 0.120880, 0.314119),Point(-0.219417, 0.126045, 0.311167),Point(-0.219191, 0.129488, 0.309044),Point(-0.219096, 0.133448, 0.306346),Point(-0.219339, 0.136374, 0.303866),Point(-0.219451, 0.138784, 0.301960),Point(-0.219762, 0.141194, 0.299744),Point(-0.220248, 0.144293, 0.296761),Point(-0.220798, 0.145671, 0.294918),Point(-0.221785, 0.148769, 0.291159),Point(-0.222940, 0.151352, 0.287509),Point(-0.224540, 0.154105, 0.283046),Point(-0.226164, 0.156172, 0.279038),Point(-0.227989, 0.158237, 0.274720),Point(-0.229951, 0.159270, 0.270931),Point(-0.232557, 0.160475, 0.266017),Point(-0.234889, 0.160992, 0.262026),Point(-0.284381, 0.160992, 0.185173),Point(-0.295157, 0.053087, 0.245991),Point(-0.296086, 0.053784, 0.244047),Point(-0.297312, 0.054282, 0.241785),Point(-0.344209, 0.053087, 0.169822),Point(-0.298315, 0.054581, 0.240014),Point(-0.342822, 0.053784, 0.171473),Point(-0.341271, 0.054282, 0.173525),Point(-0.299928, 0.054780, 0.237366),Point(-0.340073, 0.054581, 0.175170),Point(-0.301375, 0.054780, 0.235119),Point(-0.338329, 0.054780, 0.177734),Point(-0.336882, 0.054780, 0.179981),Point(-0.319128, 0.054780, 0.207550),Point(-0.210033, 0.110733, 0.309518),Point(-0.210784, 0.106601, 0.311320),Point(-0.200713, 0.101942, 0.304835),Point(-0.199962, 0.106072, 0.303033),Point(-0.273406, 0.038085, 0.263321),Point(-0.271132, 0.037396, 0.267349),Point(-0.249588, 0.043937, 0.296101),Point(-0.246812, 0.046004, 0.298927),Point(-0.286548, 0.049300, 0.234853),Point(-0.285322, 0.048802, 0.237115),Point(-0.258010, 0.039290, 0.286363),Point(-0.255215, 0.040495, 0.289837),Point(-0.208685, 0.133802, 0.295028),Point(-0.208997, 0.136212, 0.292812),Point(-0.231979, 0.060465, 0.311566),Point(-0.229353, 0.063908, 0.313171),Point(-0.213181, 0.097133, 0.314402),Point(-0.211848, 0.102126, 0.312884),Point(-0.208426, 0.124505, 0.302112),Point(-0.208331, 0.128465, 0.299414),Point(-0.350986, 0.038085, 0.142851),Point(-0.348348, 0.039117, 0.146205),Point(-0.213774, 0.149124, 0.276114),Point(-0.215400, 0.151190, 0.272106),Point(-0.330507, 0.049300, 0.166593),Point(-0.329308, 0.049600, 0.168238),Point(-0.365595, 0.039290, 0.119301),Point(-0.363532, 0.038256, 0.123246),Point(-0.377771, 0.060465, 0.085175),Point(-0.377284, 0.057366, 0.088158),Point(-0.342547, 0.041872, 0.153235),Point(-0.339427, 0.043766, 0.156719),Point(-0.372575, 0.097133, 0.066888),Point(-0.374052, 0.092313, 0.068059),Point(-0.337963, 0.149124, 0.083269),Point(-0.341365, 0.146369, 0.079967),Point(-0.376705, 0.082673, 0.070868),Point(-0.377332, 0.079229, 0.072369),Point(-0.359419, 0.124505, 0.067644),Point(-0.361446, 0.121063, 0.066972),Point(-0.325777, 0.155493, 0.097614),Point(-0.329172, 0.154289, 0.093208),Point(-0.224125, 0.156010, 0.255094),Point(-0.273616, 0.156010, 0.178241),Point(-0.260748, 0.038256, 0.282854),Point(-0.281249, 0.043766, 0.247059),Point(-0.279368, 0.041872, 0.251341),Point(-0.208653, 0.121063, 0.304235),Point(-0.243153, 0.048930, 0.302504),Point(-0.239895, 0.051857, 0.305459),Point(-0.220139, 0.079229, 0.316465),Point(-0.218513, 0.082673, 0.316516),Point(-0.268944, 0.037052, 0.270993),Point(-0.266031, 0.037396, 0.275269),Point(-0.209484, 0.139311, 0.289829),Point(-0.210034, 0.140688, 0.287986),Point(-0.219187, 0.154289, 0.263999),Point(-0.221793, 0.155493, 0.259085),Point(-0.337019, 0.045316, 0.159344),Point(-0.374143, 0.048930, 0.099098),Point(-0.372400, 0.046004, 0.103908),Point(-0.333444, 0.048105, 0.162890),Point(-0.332058, 0.048802, 0.164541),Point(-0.377859, 0.075786, 0.074027),Point(-0.356125, 0.037052, 0.135614),Point(-0.353712, 0.037396, 0.139113),Point(-0.348675, 0.139311, 0.073689),Point(-0.351188, 0.136212, 0.072011),Point(-0.378331, 0.067006, 0.079604),Point(-0.378144, 0.063908, 0.082120),Point(-0.323108, 0.156010, 0.101388),Point(-0.368777, 0.106601, 0.065981),Point(-0.370641, 0.102126, 0.066303),Point(-0.331810, 0.153255, 0.089854),Point(-0.334987, 0.151190, 0.086406),Point(-0.284393, 0.048105, 0.239059),Point(-0.252194, 0.042388, 0.293165),Point(-0.227139, 0.067006, 0.314382),Point(-0.277475, 0.040323, 0.255394),Point(-0.275369, 0.039117, 0.259531),Point(-0.217224, 0.153255, 0.267788),Point(-0.216535, 0.087493, 0.316123),Point(-0.214859, 0.092313, 0.315262),Point(-0.208574, 0.131392, 0.296934),Point(-0.327565, 0.049798, 0.170802),Point(-0.326118, 0.049798, 0.173049),Point(-0.345454, 0.040323, 0.149834),Point(-0.367601, 0.040495, 0.115319),Point(-0.375529, 0.087493, 0.069230),Point(-0.356924, 0.128465, 0.068674),Point(-0.376541, 0.054439, 0.091415),Point(-0.375485, 0.051857, 0.094910),Point(-0.344210, 0.143787, 0.077405),Point(-0.347224, 0.140688, 0.074952),Point(-0.290610, 0.049798, 0.228187),Point(-0.289163, 0.049798, 0.230434),Point(-0.237150, 0.054439, 0.307866),Point(-0.283434, 0.046610, 0.241621),Point(-0.282642, 0.045316, 0.243781),Point(-0.263218, 0.037740, 0.279390),Point(-0.224543, 0.070966, 0.315566),Point(-0.221865, 0.075786, 0.316259),Point(-0.308364, 0.049798, 0.200618),Point(-0.209142, 0.115898, 0.307187),Point(-0.211021, 0.143787, 0.284227),Point(-0.335380, 0.046610, 0.160958),Point(-0.358813, 0.037396, 0.131192),Point(-0.378336, 0.070966, 0.076750),Point(-0.366825, 0.110733, 0.066043),Point(-0.370976, 0.043937, 0.107605),Point(-0.369382, 0.042388, 0.111193),Point(-0.353078, 0.133802, 0.070812),Point(-0.354766, 0.131392, 0.069921),Point(-0.287551, 0.049600, 0.233082),Point(-0.234493, 0.057366, 0.309890),Point(-0.212175, 0.146369, 0.280577),Point(-0.361401, 0.037740, 0.126927),Point(-0.364335, 0.115898, 0.066196),Point(-0.315707, 0.150832, 0.091129),Point(-0.313038, 0.151349, 0.094903),Point(-0.319102, 0.149628, 0.086724),Point(-0.263546, 0.151349, 0.171756),Point(-0.214054, 0.151349, 0.248609),Point(-0.211723, 0.150832, 0.252601),Point(-0.209117, 0.149628, 0.257514),Point(-0.207154, 0.148594, 0.261304),Point(-0.205330, 0.146529, 0.265622),Point(-0.203705, 0.144463, 0.269629),Point(-0.202106, 0.141709, 0.274092),Point(-0.200950, 0.139126, 0.277742),Point(-0.199964, 0.136028, 0.281501),Point(-0.199414, 0.134650, 0.283345),Point(-0.198927, 0.131551, 0.286327),Point(-0.198616, 0.129141, 0.288543),Point(-0.198504, 0.126731, 0.290449),Point(-0.198261, 0.123805, 0.292929),Point(-0.198357, 0.119844, 0.295627),Point(-0.198584, 0.116402, 0.297750),Point(-0.199072, 0.111237, 0.300702),Point(-0.201779, 0.097465, 0.306399),Point(-0.203111, 0.092472, 0.307917),Point(-0.204788, 0.087652, 0.308777),Point(-0.206466, 0.082832, 0.309638),Point(-0.208443, 0.078012, 0.310032),Point(-0.210069, 0.074568, 0.309981),Point(-0.211796, 0.071125, 0.309774),Point(-0.214472, 0.066305, 0.309081),Point(-0.217068, 0.062346, 0.307897),Point(-0.219282, 0.059247, 0.306686),Point(-0.221909, 0.055804, 0.305082),Point(-0.224423, 0.052705, 0.303405),Point(-0.227080, 0.049779, 0.301381),Point(-0.229826, 0.047197, 0.298974),Point(-0.233084, 0.044269, 0.296019),Point(-0.236742, 0.041343, 0.292442),Point(-0.239517, 0.039278, 0.289616),Point(-0.242125, 0.037727, 0.286681),Point(-0.245145, 0.035834, 0.283352),Point(-0.247940, 0.034629, 0.279879),Point(-0.250679, 0.033596, 0.276369),Point(-0.253147, 0.033080, 0.272905),Point(-0.255960, 0.032736, 0.268785),Point(-0.258873, 0.032391, 0.264508),Point(-0.261061, 0.032736, 0.260864),Point(-0.263337, 0.033424, 0.256836),Point(-0.265298, 0.034456, 0.253046),Point(-0.267405, 0.035662, 0.248910),Point(-0.269298, 0.037211, 0.244856),Point(-0.271179, 0.039105, 0.240574),Point(-0.272573, 0.040655, 0.237297),Point(-0.273365, 0.041949, 0.235136),Point(-0.325310, 0.041949, 0.154473),Point(-0.326949, 0.040655, 0.152859),Point(-0.329357, 0.039105, 0.150234),Point(-0.332477, 0.037211, 0.146751),Point(-0.335384, 0.035662, 0.143349),Point(-0.338278, 0.034456, 0.139721),Point(-0.340916, 0.033424, 0.136366),Point(-0.343643, 0.032736, 0.132629),Point(-0.346055, 0.032391, 0.129129),Point(-0.348744, 0.032736, 0.124708),Point(-0.351331, 0.033080, 0.120442),Point(-0.353463, 0.033596, 0.116761),Point(-0.355525, 0.034629, 0.112816),Point(-0.357532, 0.035834, 0.108834),Point(-0.359312, 0.037727, 0.104708),Point(-0.360906, 0.039278, 0.101120),Point(-0.362330, 0.041343, 0.097423),Point(-0.364072, 0.044269, 0.092613),Point(-0.365416, 0.047197, 0.088425),Point(-0.366472, 0.049779, 0.084930),Point(-0.367214, 0.052705, 0.081673),Point(-0.367701, 0.055804, 0.078690),Point(-0.368074, 0.059247, 0.075635),Point(-0.368261, 0.062346, 0.073118),Point(-0.368266, 0.066305, 0.070266),Point(-0.367789, 0.071125, 0.067542),Point(-0.367262, 0.074568, 0.065884),Point(-0.366636, 0.078012, 0.064383),Point(-0.365459, 0.082832, 0.062746),Point(-0.363982, 0.087652, 0.061574),Point(-0.362505, 0.092472, 0.060403),Point(-0.360572, 0.097465, 0.059818),Point(-0.358707, 0.101942, 0.059496),Point(-0.356755, 0.106072, 0.059558),Point(-0.354265, 0.111237, 0.059711),Point(-0.351376, 0.116402, 0.060488),Point(-0.349349, 0.119844, 0.061159),Point(-0.346853, 0.123805, 0.062189),Point(-0.344696, 0.126731, 0.063436),Point(-0.343007, 0.129141, 0.064327),Point(-0.341119, 0.131551, 0.065527),Point(-0.338605, 0.134650, 0.067204),Point(-0.337153, 0.136028, 0.068467),Point(-0.334139, 0.139126, 0.070920),Point(-0.331295, 0.141709, 0.073482),Point(-0.327893, 0.144463, 0.076785),Point(-0.324916, 0.146529, 0.079922),Point(-0.321740, 0.148594, 0.083369),Point(-0.274323, 0.043444, 0.232575),Point(-0.323374, 0.043444, 0.156405),Point(-0.321989, 0.044141, 0.158056),Point(-0.320436, 0.044639, 0.160108),Point(-0.275252, 0.044141, 0.230630),Point(-0.319239, 0.044939, 0.161753),Point(-0.276479, 0.044639, 0.228368),Point(-0.277480, 0.044939, 0.226598),Point(-0.317496, 0.045137, 0.164317),Point(-0.279093, 0.045137, 0.223949),Point(-0.280540, 0.045137, 0.221702),Point(-0.298294, 0.045137, 0.194133),Point(-0.316049, 0.045137, 0.166564)]

In [ ]:
P = [Point(-0.336541, 0.160475, 0.104546),Point(-0.339937, 0.159270, 0.100140),Point(-0.333872, 0.160992, 0.108320),Point(-0.342575, 0.158237, 0.096786),Point(-0.345751, 0.156172, 0.093338),Point(-0.348727, 0.154105, 0.090201),Point(-0.352129, 0.151352, 0.086899),Point(-0.354974, 0.148769, 0.084337),Point(-0.357988, 0.145671, 0.081884),Point(-0.359439, 0.144293, 0.080621),Point(-0.361952, 0.141194, 0.078944),Point(-0.363842, 0.138784, 0.077743),Point(-0.365530, 0.136374, 0.076853),Point(-0.367688, 0.133448, 0.075606),Point(-0.370184, 0.129488, 0.074576),Point(-0.372210, 0.126045, 0.073904),Point(-0.375100, 0.120880, 0.073129),Point(-0.377590, 0.115715, 0.072974),Point(-0.379541, 0.111584, 0.072913),Point(-0.381406, 0.107108, 0.073235),Point(-0.383339, 0.102115, 0.073821),Point(-0.384817, 0.097295, 0.074991),Point(-0.386293, 0.092475, 0.076162),Point(-0.387471, 0.087655, 0.077799),Point(-0.388096, 0.084211, 0.079301),Point(-0.388623, 0.080768, 0.080958),Point(-0.389100, 0.075948, 0.083682),Point(-0.389095, 0.071988, 0.086536),Point(-0.388908, 0.068890, 0.089052),Point(-0.388535, 0.065446, 0.092107),Point(-0.388048, 0.062348, 0.095090),Point(-0.387305, 0.059422, 0.098347),Point(-0.386249, 0.056839, 0.101842),Point(-0.384907, 0.053912, 0.106030),Point(-0.383164, 0.050986, 0.110840),Point(-0.381740, 0.048921, 0.114537),Point(-0.380146, 0.047370, 0.118125),Point(-0.378365, 0.045477, 0.122251),Point(-0.376359, 0.044272, 0.126233),Point(-0.374296, 0.043240, 0.130178),Point(-0.372166, 0.042722, 0.133859),Point(-0.369577, 0.042379, 0.138125),Point(-0.366890, 0.042034, 0.142546),Point(-0.364477, 0.042379, 0.146045),Point(-0.361751, 0.043067, 0.149783),Point(-0.359113, 0.044099, 0.153137),Point(-0.356218, 0.045305, 0.156766),Point(-0.353311, 0.046854, 0.160167),Point(-0.350191, 0.048748, 0.163651),Point(-0.347783, 0.050297, 0.166276),Point(-0.346144, 0.051592, 0.167890),Point(-0.294198, 0.051592, 0.248553),Point(-0.293408, 0.050297, 0.250714),Point(-0.292014, 0.048748, 0.253991),Point(-0.290133, 0.046854, 0.258273),Point(-0.288239, 0.045305, 0.262326),Point(-0.286133, 0.044099, 0.266463),Point(-0.284170, 0.043067, 0.270253),Point(-0.281896, 0.042379, 0.274281),Point(-0.279708, 0.042034, 0.277925),Point(-0.276795, 0.042379, 0.282202),Point(-0.273982, 0.042722, 0.286322),Point(-0.271512, 0.043240, 0.289786),Point(-0.268774, 0.044272, 0.293295),Point(-0.265980, 0.045477, 0.296769),Point(-0.262960, 0.047370, 0.300098),Point(-0.260352, 0.048921, 0.303033),Point(-0.257576, 0.050986, 0.305859),Point(-0.253919, 0.053912, 0.309436),Point(-0.250661, 0.056839, 0.312391),Point(-0.247915, 0.059422, 0.314798),Point(-0.245257, 0.062348, 0.316822),Point(-0.242743, 0.065446, 0.318498),Point(-0.240117, 0.068890, 0.320102),Point(-0.237903, 0.071988, 0.321314),Point(-0.235307, 0.075948, 0.322498),Point(-0.232629, 0.080768, 0.323191),Point(-0.230903, 0.084211, 0.323398),Point(-0.229277, 0.087655, 0.323449),Point(-0.227299, 0.092475, 0.323054),Point(-0.225623, 0.097295, 0.322194),Point(-0.223945, 0.102115, 0.321334),Point(-0.222612, 0.107108, 0.319816),Point(-0.221548, 0.111584, 0.318252),Point(-0.220797, 0.115715, 0.316450),Point(-0.219907, 0.120880, 0.314119),Point(-0.219417, 0.126045, 0.311167),Point(-0.219191, 0.129488, 0.309044),Point(-0.219096, 0.133448, 0.306346),Point(-0.219339, 0.136374, 0.303866),Point(-0.219451, 0.138784, 0.301960),Point(-0.219762, 0.141194, 0.299744),Point(-0.220248, 0.144293, 0.296761),Point(-0.220798, 0.145671, 0.294918),Point(-0.221785, 0.148769, 0.291159),Point(-0.222940, 0.151352, 0.287509),Point(-0.224540, 0.154105, 0.283046),Point(-0.226164, 0.156172, 0.279038),Point(-0.227989, 0.158237, 0.274720),Point(-0.229951, 0.159270, 0.270931),Point(-0.232557, 0.160475, 0.266017),Point(-0.234889, 0.160992, 0.262026),Point(-0.284381, 0.160992, 0.185173),Point(-0.295157, 0.053087, 0.245991),Point(-0.296086, 0.053784, 0.244047),Point(-0.297312, 0.054282, 0.241785),Point(-0.344209, 0.053087, 0.169822),Point(-0.298315, 0.054581, 0.240014),Point(-0.342822, 0.053784, 0.171473),Point(-0.341271, 0.054282, 0.173525),Point(-0.299928, 0.054780, 0.237366),Point(-0.340073, 0.054581, 0.175170),Point(-0.301375, 0.054780, 0.235119),Point(-0.338329, 0.054780, 0.177734),Point(-0.336882, 0.054780, 0.179981),Point(-0.319128, 0.054780, 0.207550),Point(-0.210033, 0.110733, 0.309518),Point(-0.210784, 0.106601, 0.311320),Point(-0.200713, 0.101942, 0.304835),Point(-0.199962, 0.106072, 0.303033),Point(-0.273406, 0.038085, 0.263321),Point(-0.271132, 0.037396, 0.267349),Point(-0.249588, 0.043937, 0.296101),Point(-0.246812, 0.046004, 0.298927),Point(-0.286548, 0.049300, 0.234853),Point(-0.285322, 0.048802, 0.237115),Point(-0.258010, 0.039290, 0.286363),Point(-0.255215, 0.040495, 0.289837),Point(-0.208685, 0.133802, 0.295028),Point(-0.208997, 0.136212, 0.292812),Point(-0.231979, 0.060465, 0.311566),Point(-0.229353, 0.063908, 0.313171),Point(-0.213181, 0.097133, 0.314402),Point(-0.211848, 0.102126, 0.312884),Point(-0.208426, 0.124505, 0.302112),Point(-0.208331, 0.128465, 0.299414),Point(-0.350986, 0.038085, 0.142851),Point(-0.348348, 0.039117, 0.146205),Point(-0.213774, 0.149124, 0.276114),Point(-0.215400, 0.151190, 0.272106),Point(-0.330507, 0.049300, 0.166593),Point(-0.329308, 0.049600, 0.168238),Point(-0.365595, 0.039290, 0.119301),Point(-0.363532, 0.038256, 0.123246),Point(-0.377771, 0.060465, 0.085175),Point(-0.377284, 0.057366, 0.088158),Point(-0.342547, 0.041872, 0.153235),Point(-0.339427, 0.043766, 0.156719),Point(-0.372575, 0.097133, 0.066888),Point(-0.374052, 0.092313, 0.068059),Point(-0.337963, 0.149124, 0.083269),Point(-0.341365, 0.146369, 0.079967),Point(-0.376705, 0.082673, 0.070868),Point(-0.377332, 0.079229, 0.072369),Point(-0.359419, 0.124505, 0.067644),Point(-0.361446, 0.121063, 0.066972),Point(-0.325777, 0.155493, 0.097614),Point(-0.329172, 0.154289, 0.093208),Point(-0.224125, 0.156010, 0.255094),Point(-0.273616, 0.156010, 0.178241),Point(-0.260748, 0.038256, 0.282854),Point(-0.281249, 0.043766, 0.247059),Point(-0.279368, 0.041872, 0.251341),Point(-0.208653, 0.121063, 0.304235),Point(-0.243153, 0.048930, 0.302504),Point(-0.239895, 0.051857, 0.305459),Point(-0.220139, 0.079229, 0.316465),Point(-0.218513, 0.082673, 0.316516),Point(-0.268944, 0.037052, 0.270993),Point(-0.266031, 0.037396, 0.275269),Point(-0.209484, 0.139311, 0.289829),Point(-0.210034, 0.140688, 0.287986),Point(-0.219187, 0.154289, 0.263999),Point(-0.221793, 0.155493, 0.259085),Point(-0.337019, 0.045316, 0.159344),Point(-0.374143, 0.048930, 0.099098),Point(-0.372400, 0.046004, 0.103908),Point(-0.333444, 0.048105, 0.162890),Point(-0.332058, 0.048802, 0.164541),Point(-0.377859, 0.075786, 0.074027),Point(-0.356125, 0.037052, 0.135614),Point(-0.353712, 0.037396, 0.139113),Point(-0.348675, 0.139311, 0.073689),Point(-0.351188, 0.136212, 0.072011),Point(-0.378331, 0.067006, 0.079604),Point(-0.378144, 0.063908, 0.082120),Point(-0.323108, 0.156010, 0.101388),Point(-0.368777, 0.106601, 0.065981),Point(-0.370641, 0.102126, 0.066303),Point(-0.331810, 0.153255, 0.089854),Point(-0.334987, 0.151190, 0.086406),Point(-0.284393, 0.048105, 0.239059),Point(-0.252194, 0.042388, 0.293165),Point(-0.227139, 0.067006, 0.314382),Point(-0.277475, 0.040323, 0.255394),Point(-0.275369, 0.039117, 0.259531),Point(-0.217224, 0.153255, 0.267788),Point(-0.216535, 0.087493, 0.316123),Point(-0.214859, 0.092313, 0.315262),Point(-0.208574, 0.131392, 0.296934),Point(-0.327565, 0.049798, 0.170802),Point(-0.326118, 0.049798, 0.173049),Point(-0.345454, 0.040323, 0.149834),Point(-0.367601, 0.040495, 0.115319),Point(-0.375529, 0.087493, 0.069230),Point(-0.356924, 0.128465, 0.068674),Point(-0.376541, 0.054439, 0.091415),Point(-0.375485, 0.051857, 0.094910),Point(-0.344210, 0.143787, 0.077405),Point(-0.347224, 0.140688, 0.074952),Point(-0.290610, 0.049798, 0.228187),Point(-0.289163, 0.049798, 0.230434),Point(-0.237150, 0.054439, 0.307866),Point(-0.283434, 0.046610, 0.241621),Point(-0.282642, 0.045316, 0.243781),Point(-0.263218, 0.037740, 0.279390),Point(-0.224543, 0.070966, 0.315566),Point(-0.221865, 0.075786, 0.316259),Point(-0.308364, 0.049798, 0.200618),Point(-0.209142, 0.115898, 0.307187),Point(-0.211021, 0.143787, 0.284227),Point(-0.335380, 0.046610, 0.160958),Point(-0.358813, 0.037396, 0.131192),Point(-0.378336, 0.070966, 0.076750),Point(-0.366825, 0.110733, 0.066043),Point(-0.370976, 0.043937, 0.107605),Point(-0.369382, 0.042388, 0.111193),Point(-0.353078, 0.133802, 0.070812),Point(-0.354766, 0.131392, 0.069921),Point(-0.287551, 0.049600, 0.233082),Point(-0.234493, 0.057366, 0.309890),Point(-0.212175, 0.146369, 0.280577),Point(-0.361401, 0.037740, 0.126927),Point(-0.364335, 0.115898, 0.066196),Point(-0.315707, 0.150832, 0.091129),Point(-0.313038, 0.151349, 0.094903),Point(-0.319102, 0.149628, 0.086724),Point(-0.263546, 0.151349, 0.171756),Point(-0.214054, 0.151349, 0.248609),Point(-0.211723, 0.150832, 0.252601),Point(-0.209117, 0.149628, 0.257514),Point(-0.207154, 0.148594, 0.261304),Point(-0.205330, 0.146529, 0.265622),Point(-0.203705, 0.144463, 0.269629),Point(-0.202106, 0.141709, 0.274092),Point(-0.200950, 0.139126, 0.277742),Point(-0.199964, 0.136028, 0.281501),Point(-0.199414, 0.134650, 0.283345),Point(-0.198927, 0.131551, 0.286327),Point(-0.198616, 0.129141, 0.288543),Point(-0.198504, 0.126731, 0.290449),Point(-0.198261, 0.123805, 0.292929),Point(-0.198357, 0.119844, 0.295627)]

In [ ]:
H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

In [1]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            try:
                points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                return points
    return points

In [2]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')

In [3]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for point in P:
        print("v {} {} {}".format(point.x, point.y, point.z))
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [24]:
esfera = read_vertices_from_obj(os.getcwd()+"\\obj\\esfera.obj")
print(len(esfera))
HullEsfera = GiftWrap(esfera)
write_faces(HullEsfera, esfera, "hullEsfera.obj")

482


KeyboardInterrupt: 

In [25]:
controle = read_vertices_from_obj(os.getcwd()+"\\obj\\controle.obj")
print(len(controle))
hullControle = GiftWrap(controle)
print(len(hullControle))
write_faces(hullControle, controle, os.getcwd()+"\\obj\\hullControle.obj")

348
27664


## PB

In [ ]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ", " + str(self.z) + ")"

In [ ]:
p = Point(1, 2, 3)
print(p)

In [ ]:
class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __str__(self):
        return "[" + str(self.p1) + " -> " + str(self.p2) + "]"

In [ ]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

In [ ]:
class Face:
    def __init__(self, e1, e2, e3):
        self.e1 = e1
        self.e2 = e2
        self.e3 = e3

    def __str__(self):
        return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"

In [ ]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)

print(f)

In [ ]:
def CrossProduct(v1, v2):
    

In [ ]:
def Normalize(v):
    

In [ ]:
def SquaredArea(p1, p2, p3):
    

In [ ]:
def SignedVolume(p1, p2, p3, p4):
    

In [ ]:
def PivotAroundEdge(e, P):
    p = P.pop()
    area2 = SquaredArea(e.p1, e.p2, p)

    for p_ in P:
        volume = SignedVolume(e.p1, e.p2, p, p_)
        if volume < 0:
            p = p_
        elif volume == 0:
            area2_ = SquaredArea(e.p1, e.p2, p_)
            if area2_ > area2:
                p = p_
                area2 = area2_
    return p

In [ ]:
def BottomMostLeftMostBackMost(P):
    

In [ ]:
def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = p

    for r in P:
        if (q.z == r.z) and (q.y == z.y) and (q.x < r.x):
            q = r
    
    if (q == p):
        q = Point(p.x + 1, p.y. p.z)
        q = PivotAroundEdge(Edge(p, q), P)

    return Edge(p, q)

In [ ]:
def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

In [ ]:
def GiftWrap(P):
    t = FindTriangleOnHull(P)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []

    while not len(Q) == 0:
        e = Q.pop()
        if NotProcessed(e):
            q = PivotAroundEdge(e, P)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            H.append(t)
            Q.append(t.e1, t.e2, t.e3)
            Processed.append(e)

    return H

## angulo


In [ ]:
import math
from typing import List

import matplotlib.pyplot as plt
import numpy as np


class Point():
    """Data structure of a point."""

    def __init__(self, x, y, z=0) -> None:
        """Point object

        Args:
            x (float): x coordinate of point
            y (float): y coordinate of point
        """
        self.x = x
        self.y = y
        self.z = z

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.z

    def __len__(self):
        return 3

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def to_numpy(self):
        return np.array([self.x, self.y])


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __eq__(self, other):
        # return self.__dict__ == other.__dict__
        if self.p1 == other.p1 or self.p1 == other.p2:
           if self.p2 == other.p1 or self.p2 == other.p2:
               return True
        return False
    
    def __str__(self):
        return "[" + str(self.p1) + " -> " + str(self.p2) + "]"

    def __call__(self):
        # return "[" + str(self.p1) + " -> " + str(self.p2) + "]"
        return (self.p1(), self.p2())


class Face:
    def __init__(self, e1, e2, e3):
        self.e1 = e1
        self.e2 = e2
        self.e3 = e3

    def __str__(self):
        return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"

    def __call__(self):
        # return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"
        return self.e1.p1(), self.e1.p2(), self.e3.p1()

    def __eq__(self, other):
        if self.e1 == other.e1 or self.e1 == other.e2 or self.e2 == other.e3:
            if self.e2 == other.e1 or self.e2 == other.e2 or self.e2 == other.e3:
                if self.e3 == other.e1 or self.e3 == other.e2 or self.e3 == other.e3:
                    return True
        return False
        # p1, p2, p3 = self()
        # p1_, p2_, p3_ = other()

        # if p1 == p1_ or p1 == p2_ or p1 == p3_:
        #     if p2 == p1_ or p2 == p2_ or p2 == p3_:
        #         if p3 == p1_ or p3 == p2_ or p3 == p3_:
        #             return True
        # return False


class Gcrf():

    def gen_poly(self, n):
        xn = np.random.random(n)
        yn = np.random.random(n)
        z = zip(xn, yn)
        z = [Point(x, y) for x, y in z]
        z = np.array(z)
        angles = [self.pseudo_angulo_orientado(i) for i in z]
        indexes = np.argsort(angles)
        z = z[indexes]
        z = z.tolist()

        return z

    def soma_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x+b.x, a.y+b.y, a.z+b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]+b[i])
        return z

    def subtr_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x-b.x, a.y-b.y, a.z-b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]-b[i])
        return z

    def multi_escalar(self, lamb: int, x: list) -> list:
        # if isinstance(a, Point):
        #     return Point(a.x*lamb, a.y*lamb, a.z*lamb)
        z = []
        for i in range(len(x)):
            z.append(lamb*x[i])
        return z

    def prod_escalar(self, a, b):
        aIsPoint = isinstance(a, Point)
        bIsPoint = isinstance(b, Point)
        if aIsPoint and bIsPoint:
            return sum([a.x*b.x, a.y*b.y, a.z*b.z])
        if aIsPoint:
            a = a()
        if bIsPoint:
            b = b()
        res = []
        for i in range(len(a)):
            res.append(a[i]*b[i])
        return sum(res)

    def norma(self, a):
        # print(a)
        # for i in range(len(x)):
            # norm += x[i]**2
        norm = a.x**2 + a.y**2 + a.z**2
        return math.sqrt(norm)

    def distancia(self, x, y):
        """Returns the distance between vectors x and y.

        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.

        Returns:
            [float]: [the distance between vectors x and y]
        """
        z = []
        for i in range(len(x)):
            z.append(x[i]-y[i])
        return self.norma(z)

    def angulo(self, x, y, degrees=False):
        """Returns the angle between vectors x and y.

        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.
            degrees (bool): if the result should be in degrees or radians, defaults to False.
        Returns:
            float: the angle between the two vectors
        """

        # if not degrees:
        #     return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        # return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))*(180/math.pi)
        # print(x, y)
        if not degrees:
            return self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))
        return self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))*(180/math.pi)

    def angulo_orientado(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*math.acos(x[0]/self.norma(x))
        return mult*math.acos(x[0]/self.norma(x))*(180/math.pi)

    def pseudo_angulo(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*(1-(x[0]/self.norma(x)))
        return mult*(1-(x[0]/self.norma(x)))*(180/math.pi)

    def pseudo_angulo_cosseno(self, x, y=[0, 0], degrees=False):
        if isinstance(x, Point):
            x = x()
        if not degrees:
            return 1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        return (1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))))*(180/math.pi)

    def pseudo_angulo_orientado(self, x):
        # print("X do pseudoângulo",x)
        if isinstance(x, Point):
            x = x()
        if x[1] >= 0:
            if x[0] >= 0:
                if x[0] >= x[1]:
                    return x[1]/x[0]
                return 2-(x[0]/x[1])
            if -x[0] <= x[1]:
                return 2+(-x[0]/x[1])
            return 4-(x[1]/-x[0])
        if x[0] < 0:
            if -x[0] >= -x[1]:
                return 4+(-x[1]/-x[0])
            return 6-(-x[0]/-x[1])
        if x[0] <= -x[1]:
            return 6+(x[0]/-x[1])
        return 8-(-x[1]/x[0])

    def pseudo_angulo_2(self, x, y):
        return self.pseudo_angulo_orientado(y)-self.pseudo_angulo_orientado(x)

    def prod_vetorial(self, a, b):
        if len(a) == 2:
            return a.x*b.y-a.y*b.x
        return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

    def intersect(self, a, b, c, d):
        ab = self.subtr_vetorial(b, a)
        ac = self.subtr_vetorial(c, a)
        ad = self.subtr_vetorial(d, a)
        ca = self.subtr_vetorial(a, c)
        cb = self.subtr_vetorial(b, c)
        cd = self.subtr_vetorial(d, c)

        # r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad)
        r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad) < 0
        # r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb)
        r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb) < 0
        # return (r1, r2), r1 < 0 and r2 < 0
        return r1 and r2

    def square_area(self, p1: Point, p2: Point, p3: Point):
        res = self.prod_vetorial(self.subtr_vetorial(
            p2, p1), self.subtr_vetorial(p3, p1))
        res = sum([i**2 for i in res])
        return 0.5*res

    def area(self, x, y):
        if len(x) == 2:
            return abs(self.prod_vetorial(x, y))
        return self.norma(self.prod_vetorial(x, y))

    def signed_volume(self, p1: Point, p2: Point, p3: Point, p4: Point):
        v1 = self.subtr_vetorial(p2, p1)
        v2 = self.subtr_vetorial(p3, p1)
        v3 = self.subtr_vetorial(p4, p1)
        res = self.prod_vetorial(v1, v2)
        return self.prod_escalar(v3, res)/6

    def antiHorario(self, listaDePontos):
        res = 0
        for i in range(len(listaDePontos)-1):
            res += self.prod_vetorial(listaDePontos[i], listaDePontos[i+1])
        res += self.prod_vetorial(listaDePontos[0], listaDePontos[-1])
        return 0.5*res > 0

    def slope(self, p1: Point, p2: Point):
        return (p2.y - p1.y) / (p2.x - p1.x)

    def pip(self, p: Point, poly) -> bool:
        """ Determine if the point is in the polygon.

        Args:
            p -- a object of type Point(x, y)
            poly -- a list of tuples [(x, y), (x, y), ...]

        Returns:
            True if the point is in the path or is a corner or on the boundary"""

        num = len(poly)
        j = num - 1
        result = False
        for i in range(len(poly)):
            if (p.x == poly[i].x) and (p.y == poly[i].y):
                return True
            if ((poly[i].y > p.y) != (poly[j].y > p.y)):
                slope = self.slope(p, poly[i])
                if slope == 0:
                    return True
                if (slope < 0) != (poly[j].y < poly[i].y):
                    result = not result
            j = i
        return result

    def starPolygon(self, points, jumps):
        i = 0
        starredPolygon = []
        while i < jumps:
            for j in range(i, len(points), jumps):
                # print("Ponto adicionado j:",points[j]())
                starredPolygon.append(points[j])
            # print("Ponto adicionado i:",points[i]())
            starredPolygon.append(points[i])
            i += 1
        return starredPolygon

    def centeroidnp(self, points):
        length = len(points)
        tipoPoints = type(points[0])

        if tipoPoints == tuple:
            arr = np.array(points)
        elif tipoPoints == Point:
            arr = np.array([p.to_numpy() for p in points])

        sum_x = np.sum(arr[:, 0])
        sum_y = np.sum(arr[:, 1])
        return sum_x/length, sum_y/length

    def plotPoint(self, p: Point):
        plt.scatter(p.x, p.y)
        plt.show()

    def plotPoints(self, p: List[Point]):
        for point in p:
            self.plotPoint(point)
        plt.show()

    def plotPolygon(self, points, annotate=False):
        """Plot polygon from list of points

        Args:
            points (Point): list of Point objects indicating the polygon vertices
        """

        points.append(points[0])
        if type(points[0]) == Point:
            points = [p() for p in points]
        x, y = zip(*points)
        plt.grid(0.5)
        plt.plot(x, y, '-o')
        if annotate:
            plt.text(x[0], y[0], "p1")
            plt.text(x[1], y[1], "p2")
            plt.text(x[2], y[2], "p3")
        plt.show()

    def plotSegment(self, segments: List[Point], threeD=False) -> None:
        if threeD:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

        for segment in segments:
            segx = [p.x for p in segment]
            segy = [p.y for p in segment]
            if threeD:
                segz = [p.z for p in segment]
                ax.plot(segx, segy, segz)
            else:
                plt.plot(segx, segy)
        plt.show()

    def plotSquare(self):
        x = [1, 1, -1, -1, 1]
        y = [1, -1, -1, 1, 1]
        plt.plot(x, y)

    def plotAxis(self):
        plt.grid()
        plt.axhline(y=0, color='k')
        plt.axvline(x=0, color='k')


    # def plot_faces(self, F):


    def rotationIndex(self, p: Point, poly: list):
        # NOT WORKING
        j = len(poly)-1
        rotIndex = 0
        for i in range(len(poly)):
            ppi = self.subtr_vetorial(p, poly[j])
            ppi1 = self.subtr_vetorial(p, poly[i])
            rotIndex += self.pseudo_angulo_2(ppi, ppi1)
            j = i
        rotIndex *= 1/(2*math.pi)
        if rotIndex == 0:
            return False
        if abs(rotIndex) == 1:
            return True
        return rotIndex

In [ ]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False
    
def PivotAroundEdge(f, e, P):
    minAngle = 2*math.pi
    candidate = None
    v1 = f.e1.p2 - f.e1.p1
    v2 = f.e2.p2 - f.e1.p1
    normal = gc.prod_vetorial(v1, v2)
#     print("NORMAL:", normal)
    for i in range(len(P)):
#         print("P[i]",P[i])
        v1 = e.p2 - e.p1
        v2 = P[i] - e.p1
        n_ = gc.prod_vetorial(v1, v2)
        angle = gc.angulo(normal, n_)
        if angle < minAngle:
            minAngle = angle
            candidate = i
    return P[candidate]

def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = Point(p.x + 1, p.y, p.z)
    q = PivotAroundEdge(Edge(p, q), P)
    
    return Edge(p, q)

def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True



In [ ]:
gc = Gcrf()

def GiftWrap(P):
    t = FindTriangleOnHull(P)
#     print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []

    while not len(Q) == 0:
        e = Q.pop(0)
        #e = Edge(e.p2, e.p1)
        #print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            #print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P)
            #print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            #print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
                #print("{} entrou no Hull".format(t))
#             e_ = Edge(e.p2, e.p1)
            Q.extend([t.e1, t.e2, t.e3])
            #print("Q agora possui as arestas:")
            #for q in Q: print(q)
            Processed.append(e)
            #print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
            #for aresta in Processed: print(aresta)
        #else: print("Aresta JÁ PROCESSADA")
    return H

In [ ]:
p1 = Point(0, 2, 0)
p2 = Point(0, 0, 5)
p3 = Point(0, 0, -10)
p4 = Point(0, 0, 0)
p5 = Point(0.1, 0.1, 0.1)
p6 = Point(-3, 0, 0)
p7 = Point(0, -3, 0)
p8 = Point(0, 0, 1)
p9 = Point(0, 0, -4)
p10 = Point(4, 0, 0)

P = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))